In [8]:
import pandas as pd

df = pd.read_csv("Nat_Gas.csv")
df['Dates'] = pd.to_datetime(df['Dates'])
df = df.sort_values('Dates')
df = df.set_index('Dates')

df.head(), df.tail()


/tmp/ipython-input-1431920805.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


(            Prices
 Dates             
 2020-10-31    10.1
 2020-11-30    10.3
 2020-12-31    11.0
 2021-01-31    10.9
 2021-02-28    10.9,
             Prices
 Dates             
 2024-05-31    11.4
 2024-06-30    11.5
 2024-07-31    11.6
 2024-08-31    11.5
 2024-09-30    11.8)

In [9]:
import numpy as np

# --- create time variable ---
t0 = df.index.min()
df['t_years'] = (df.index - t0).days / 365.25

# --- fit trend on log prices ---
y = np.log(df['Prices'])
b, a = np.polyfit(df['t_years'], y, 1)

# --- build seasonality ---
df['month'] = df.index.month
overall_mean = df['Prices'].mean()
monthly_avg = df.groupby('month')['Prices'].mean()
seasonal_factor = monthly_avg / overall_mean

a, b, seasonal_factor


(np.float64(2.3323492423813126),
 np.float64(0.04190679788628937),
 month
 1     1.050675
 2     1.043983
 3     1.050675
 4     0.997137
 5     0.962338
 6     0.954753
 7     0.972599
 8     0.965907
 9     0.988214
 10    0.959215
 11    1.010522
 12    1.043983
 Name: Prices, dtype: float64)

In [12]:
def get_price(date_str):
    """
    Returns natural gas price estimate for any given date.
    Uses historical price if available, otherwise trend + seasonality.
    """
    date = pd.to_datetime(date_str)

    # If exact historical date exists, return real price
    if date in df.index:
        return float(df.loc[date, 'Prices'])

    # Otherwise estimate
    t_years = (date - t0).days / 365.25
    trend_price = np.exp(a + b * t_years)
    seasonal = seasonal_factor.loc[date.month]

    return float(trend_price * seasonal)
print(get_price("2021-06-30"))   # should match dataset
print(get_price("2025-01-31"))   # future estimate

10.0
12.935359841198668


In [14]:
def price_storage_contract(
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    max_storage_volume,
    max_injection_rate,
    max_withdrawal_rate,
    storage_cost_per_month
):
    """
    All dates as 'YYYY-MM-DD'
    Volumes in MMBtu
    Storage cost is per month
    """

    # Combine all events into one timeline
    events = []

    for d, v in zip(injection_dates, injection_volumes):
        events.append((pd.to_datetime(d), "inject", v))

    for d, v in zip(withdrawal_dates, withdrawal_volumes):
        events.append((pd.to_datetime(d), "withdraw", v))

    events.sort(key=lambda x: x[0])

    inventory = 0
    total_value = 0
    last_date = events[0][0]

    for date, action, volume in events:

        # ---- storage cost for time gap ----
        months_passed = (date.year - last_date.year) * 12 + (date.month - last_date.month)
        if months_passed > 0:
            total_value -= months_passed * storage_cost_per_month

        price = get_price(date)

        # ---- injection ----
        if action == "inject":
            if volume > max_injection_rate:
                raise ValueError("Injection exceeds max injection rate")
            if inventory + volume > max_storage_volume:
                raise ValueError("Storage capacity exceeded")

            total_value -= price * volume
            inventory += volume

        # ---- withdrawal ----
        if action == "withdraw":
            if volume > max_withdrawal_rate:
                raise ValueError("Withdrawal exceeds max withdrawal rate")
            if volume > inventory:
                raise ValueError("Not enough gas in storage")

            total_value += price * volume
            inventory -= volume

        last_date = date

    return total_value


In [15]:
value = price_storage_contract(
    injection_dates=["2024-08-31"],
    injection_volumes=[500000],

    withdrawal_dates=["2025-01-31"],
    withdrawal_volumes=[500000],

    max_storage_volume=1000000,
    max_injection_rate=600000,
    max_withdrawal_rate=600000,
    storage_cost_per_month=20000
)

print("Simple contract value: $", round(value, 2))


Simple contract value: $ 617679.92


In [16]:
value = price_storage_contract(
    injection_dates=["2024-06-30", "2024-07-31", "2024-08-31"],
    injection_volumes=[200000, 200000, 200000],

    withdrawal_dates=["2025-01-31", "2025-02-28"],
    withdrawal_volumes=[300000, 300000],

    max_storage_volume=700000,
    max_injection_rate=250000,
    max_withdrawal_rate=350000,
    storage_cost_per_month=30000
)

print("Multi-date contract value: $", round(value, 2))


Multi-date contract value: $ 588905.89
